# 제주 핫 플레이스 지도에 표시하기


In [1]:
!apt-get install -y fonts-nanum > /dev/null
!fc-cache -fv > /dev/null
!rm -rf ~/.cache/matplotlib > /dev/null
# 런타임 다시시작

In [1]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 제주핫플레이스.csv to 제주핫플레이스.csv


In [3]:
import pandas as pd
raw_df = pd.read_csv(filename)
raw_df.head()

,content,date,like,place,tags
0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."


In [6]:
location_counts = raw_df.place.value_counts().to_frame()
location_counts

,place
Jeju,241
Jeju-do,172
Jungle Book by Alice,101
Seogwipo,65
제주도 크리스마스 박물관,59
...,...
더아트스테이호텔-The Artstay Hotel,1
금악리 제주힐링리조트,1
카페2472,1
제주 성산읍,1


In [7]:
location_counts.to_csv('location_counts.csv')

In [9]:
loc_df = pd.read_csv('location_counts.csv')
loc_df.columns = ['지명', '빈도']
loc_df.head()

,지명,빈도
0,Jeju,241
1,Jeju-do,172
2,Jungle Book by Alice,101
3,Seogwipo,65
4,제주도 크리스마스 박물관,59


## 2. 카카오검색 api를 이용한 장소검색

In [10]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving kakaoapikey.txt to kakaoapikey.txt


In [11]:
with open(filename) as f:
  api_key = f.read()

In [28]:
import requests
from urllib.parse import quote
import numpy as np
import re

In [15]:
keyword = '합정 스타벅스'
search_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
url = f'{search_url}?query={quote(keyword)}'
# "Authorization: KakaoAK {REST_API_KEY}"
result = requests.get(url,
                      headers={"Authorization": f'KakaoAK {api_key}'}).json()

In [17]:
result['documents'][0]

{'address_name': '서울 마포구 서교동 395-166',
 'category_group_code': 'CE7',
 'category_group_name': '카페',
 'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
 'distance': '',
 'id': '26572121',
 'phone': '1522-3232',
 'place_name': '스타벅스 서교점',
 'place_url': 'http://place.map.kakao.com/26572121',
 'road_address_name': '서울 마포구 양화로 78',
 'x': '126.916980454434',
 'y': '37.5514601750423'}

In [25]:
def find_place(keyword):
  url = f'{search_url}?query={quote(keyword)}'
  result = requests.get(url,
                        headers={"Authorization": f'KakaoAK {api_key}'}).json()

  try:
    name = result['documents'][0]['place_name']
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
  except:
    print(keyword)
    name = ''
    lng = np.nan
    lat = np.nan
  
  return name, lng, lat, keyword

In [26]:
find_place('제주국제공항')

('제주국제공항', 126.492769004244, 33.5070789578184, '제주국제공항')

In [32]:
re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]', '','제주도 서귀포 중문관광단지 濟州島西歸浦中文觀光團地' )

'제주도서귀포중문관광단지'

In [ ]:
# loc_df에 공식명칭, 위도, 경도 정보추가
name_list, lng_list, lat_list = [], [], []

for place in loc_df.지명:
  place = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]', '', place).strip()

  if place:
    name, lng, lat, _ = find_place(place)
  else:
    name, lng, lat = '', np.nan, np.nan
    
  name_list.append(name)
  lng_list.append(lng)
  lat_list.append(lat)

In [54]:
loc_df['공식명칭'] = name_list
loc_df['위도'] = lat_list
loc_df['경도'] = lng_list
loc_df.head()

,지명,빈도,공식명칭,위도,경도
0,Jeju,241,,NaN,NaN
1,Jeju-do,172,,NaN,NaN
2,Jungle Book by Alice,101,,NaN,NaN
3,Seogwipo,65,,NaN,NaN
4,제주도 크리스마스 박물관,59,바이나흐튼 크리스마스박물관,33.291572,126.327858


In [55]:
# Null 데이터 삭제
loc_df.isnull().sum()

지명        0
빈도        0
공식명칭      0
위도      198
경도      198
dtype: int64

In [56]:
loc_df.isna().sum()

지명        0
빈도        0
공식명칭      0
위도      198
경도      198
dtype: int64

In [57]:
# null 데이터 삭제
total_df = loc_df.dropna()
total_df.isnull().sum().sum()

0

In [ ]:
total_df.head()

### 3. 공식명칭

In [ ]:
# 공식명칭 중복 여부
total_df.공식명칭.value_counts()[:10]

In [ ]:
df = total_df.pivot_table(index=['공식명칭', '위도', '경도'], values='빈도', aggfunc='sum')
df.sort_values(by='빈도', ascending=False)

In [ ]:
df.reset_index(inplace=True)
df

In [61]:
df.to_csv('공식명칭.csv', index=False)

### 4. folium 지도 시각화

- 1. 개별 표시

In [74]:
df.sort_values(by='빈도', ascending=False)

,,,빈도
공식명칭,위도,경도,
바이나흐튼 크리스마스박물관,33.291572,126.327858,59
제주에인감귤밭,33.256561,126.539014,48
성산일출봉,33.459135,126.940538,38
폼포코식당,33.450408,126.919065,30
한라산1100고지,33.358078,126.462220,25
...,...,...,...
에이그라운드,33.415068,126.836273,1
엉장매하우스,33.545245,126.641408,1
엉또폭포,33.268532,126.499965,1


In [ ]:
df[['빈도']].boxplot()

In [76]:
import folium

In [99]:
df.head()

,index,공식명칭,위도,경도,빈도
0,0,1158족욕카페,33.467075,126.336504,1
1,1,40계단,35.103911,129.034571,1
2,2,702키친,33.498835,126.534134,1
3,3,CU 수원광장점,37.268151,127.000311,1
4,4,KT 브로 라페스타점,37.662346,126.767821,1


In [101]:
df.drop(['index'], axis=1)

,공식명칭,위도,경도,빈도
0,1158족욕카페,33.467075,126.336504,1
1,40계단,35.103911,129.034571,1
2,702키친,33.498835,126.534134,1
3,CU 수원광장점,37.268151,127.000311,1
4,KT 브로 라페스타점,37.662346,126.767821,1
...,...,...,...,...
752,휘닉스제주섭지코지 제주,33.430497,126.928187,1
753,휴애리자연생활공원,33.309410,126.635410,11
754,흑돼지가있는풍경 본점,33.475376,126.480646,1
755,흑송,35.175011,129.196513,1


In [110]:
Mt_Hanla = [33.3625, 126.533694]
map = folium.Map(location=Mt_Hanla, zoom_start=10)

for i in df.index:
  folium.CircleMarker(
      radius = int(df.빈도[i]), # int64는 처리불가
      location = [df.위도[i], df.경도[i]],
      popup = folium.Popup(df.공식명칭[i], max_width=200),
      tooltip = df.공식명칭[i],
      color = 'red',
      fill = True
  ).add_to(map)
map

- 2. 그룹으로 표시

In [108]:
from folium.plugins import MarkerCluster

locations, names = [], []

for i in df.index:
  locations.append([df.위도[i], df.경도[i]])
  names.append(df.공식명칭[i])

In [112]:
map = folium.Map(location=Mt_Hanla, zoom_start=11)

MarkerCluster(
    locations=locations, popups=names,
    name='제주', overlay=True, control=True
).add_to(map)

map

In [113]:
map.save('제주도 핫플레이스(그룹).html')